<a href="https://colab.research.google.com/github/Tanguyvans/StGeorge/blob/main/Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports 

In [ ]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

In [25]:
import sys
import urllib
import os
from matplotlib import pyplot as plt
import math

import zipfile
import pandas as pd
import csv
from google.colab import drive
from google.colab import files

import numpy as np
from PIL import Image

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.models import Model, load_model
from keras import backend as K
from keras.applications.vgg16 import VGG16, preprocess_input # 224*224
from keras.applications.xception import Xception, preprocess_input, decode_predictions # 299*299
from keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions # 224*224
from keras.applications.densenet import DenseNet121 # 224*224

from keras.preprocessing.image import ImageDataGenerator
from keras.losses import categorical_crossentropy
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Flatten, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Defining the configuration

In [ ]:
drive.mount('/content/gdrive')
!printf '%s\n' 'george' 'no_george'> classes.txt

source_path = '/content/gdrive/MyDrive/test_assignment_cv/'
dataset_path = '/content/gdrive/MyDrive/george_ds/'
test_path= '/content/gdrive/MyDrive/george_test_ds/'

In [21]:
configs = dict(
    nb_classes = 2,
    batch_size = 64,
    input_dim = 224, 
    epochs = 15, 
    dataset_name = dataset_path,
    test_name = test_path,
    classifier = "Xception",
    pretrain_weights = 'imagenet',
    init_learning_rate = 0.001,
    lr_decay_rate = 0.1, 
    optimizer = 'adam',
    loss_fn = 'categorical_crossentropy',
    metrics = ['acc'],
    seed = 42, 
    validation_split = 0.2
)

classes_path = 'classes.txt'
log_path='logs'
result_path = 'results/' + configs['classifier']

In [ ]:
with open(classes_path, 'r') as f:
    classes = f.readlines()
    classes = list(map(lambda x: x.strip(), classes))
num_classes = len(classes)

print(f'Classes : {classes}')
print(f'Number of classes : {num_classes}')

# Creating Train, validation adn test dataset

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
	configs['dataset_name'],          																		  # Path of the dataset
	validation_split = configs['validation_split'],             						# Data division : validation (20%), train (80%)
	subset = 'training',                																		# Selection of training data
	seed = configs['seed'],                          												# Initialization of random generator (for permutations)
	image_size = (configs['input_dim'], configs['input_dim']),    					# Input size of images
	batch_size = configs['batch_size'],																			# Batch_size
  label_mode = 'categorical'     																					# Conversion to One-Hot format
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
	configs['dataset_name'],          																		  # Path of the dataset
	validation_split = configs['validation_split'],             						# Data division : validation (20%), train (80%)
	subset = 'validation',              																		# Selection of validation data
	seed = configs['seed'],                         												# Initialization of random generator (for permutations)
	image_size = (configs['input_dim'], configs['input_dim']),    					# Input size of images
	batch_size = configs['batch_size'], 																		# Batch_size
  label_mode = 'categorical'     																					# Conversion to One-Hot format
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
	configs['test_name'],          																		  
	image_size = (configs['input_dim'], configs['input_dim']),    					# Input size of images
  label_mode = 'categorical'     																					# Conversion to One-Hot format
)

# Making data augmentation on the training set

In [ ]:
data_gen_args = dict(              
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=20, 
    width_shift_range=0.3, 
    height_shift_range=0.3,
    validation_split = 0.2,
    horizontal_flip = True,
    dtype = 'uint8'
)

color_datagen = ImageDataGenerator(**data_gen_args)

train_generator = color_datagen.flow_from_directory(
  configs['dataset_name'],          																		  # Path of the dataset             			
	subset = 'training',                																		# Selection of training data
	seed = configs['seed'],                         												# Initialization of random generator (for permutations)
	target_size = (configs['input_dim'], configs['input_dim']),    					# Input size of images
	batch_size = configs['batch_size'], 																		# Batch_size
  class_mode = 'categorical',
  shuffle = True
  )

# Building the model

In [18]:
def build_model():

  base_model = Xception(
      include_top = False, 
      weights = 'imagenet', 
      input_shape = (configs['input_dim'], configs['input_dim'],3))

  model = base_model.output
  model = Flatten()(model)

  # Adding new layers to the xception model
  model = Dense(128, activation='relu')(model)
  model = Dropout(0.4)(model)
  model = Dense(32, activation = 'relu')(model)
  model = Dropout(0.4)(model)
  predictions = Dense(2, activation = 'softmax')(model)

  model = Model(inputs=base_model.inputs, outputs = predictions)

  return model

In [ ]:
model = build_model()

# training the model

In [22]:
def train(config: dict, callbacks: list, verbose: int=0):
  tf.keras.backend.clear_session()                  
  model = build_model()

  # Freasing all the layers (can't update the weights)
  for layer in model.layers:
      layer.trainable = False       
  
  # unfreasing last layers to update the weihgts
  for layer in model.layers[:-6]:
      layer.trainable = True

  # choosing the optimizer usually Adam is the best
  opt = keras.optimizers.SGD(learning_rate = config['init_learning_rate'])
  opt2 = keras.optimizers.Adam(learning_rate = config['init_learning_rate'])
  opt3 = keras.optimizers.RMSprop(learning_rate = config['init_learning_rate'])

  model.compile(loss = config['loss_fn'],
                optimizer = opt2,
                metrics = config['metrics'])  

  if os.path.exists(result_path) == False:
      os.makedirs(result_path)

  history = model.fit_generator(
      train_generator,
      steps_per_epoch=math.ceil(len(train_generator)),
      epochs=config['epochs'],
      validation_data = val_ds,
      validation_steps=math.ceil(len(val_ds)),
      callbacks = callbacks
  )
  
  return model, history

In [23]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_path,
                                                      histogram_freq = 1)

# earlystopping to avoid overfitting. kepping the best model
stopper_callback = EarlyStopping(monitor = 'val_loss',
                                 patience = 5,
                                 mode='auto',
                                 restore_best_weights=True)

# saving model at chekcpoints
ckpt_save = os.path.join('.', 'model_fine_ep_{epoch}_val_acc_{val_acc:.3f}.h5')
ckpt_callbak = ModelCheckpoint(ckpt_save,
                               monitor = 'val_acc',
                               verbose = 1,
                               save_best_only = True,
                               mode = 'auto')



# Model evaluation

In [ ]:
callbacks = [tensorboard_callback,
             stopper_callback,
             ckpt_callbak]

# Start training
model, history = train(configs, callbacks, 1)

# evaluate the model on the validation data
loss, acc = model.evaluate(val_ds)
print(f'Validation loss: {loss}, validation accuracy : {acc}')

# evaluate the mode lon the test data
loss, acc = model.evaluate(test_ds)
print(f'Validation loss: {loss}, validation accuracy : {acc}')

# saving the model
model.save('Xception_64_15.h5')

# Making a prediction

In [ ]:
model = load_model('Xception_64_15.h5')

# upload an image that we want to classify
files.upload()

In [ ]:
classes = train_ds.class_names
image_path = "/content/img.jpg"
img = Image.open(image_path).convert('RGB')
x = tf.keras.utils.img_to_array(img,data_format='channels_last')
x = tf.keras.preprocessing.image.smart_resize(x, size=(configs['input_dim'], configs['input_dim']))
x = np.expand_dims(x, axis=0)

# predict
pred = model.predict(x,batch_size=1)[0]


print("THE prediction is: ",classes[np.argmax(pred)])